<a href="https://colab.research.google.com/github/samantha-isaac/Emotion-Detection-in-Images-and-Music-Pairing-Through-AI/blob/main/CNN_SVM_FTuning_Landscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNs with SVMs Grid Search for Landscape Dataset

Samantha Isaac

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import os
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# This is to mount Drive to this project
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path to the folder for the Landscape dataset
dataset_folder = '/content/drive/MyDrive/Dissertation/Code/Data_Landscapes'

In [ ]:
# This function helps load the images and assigns as the labels the name of the correspodning folder
# References used for the function with os: https://docs.python.org/3/library/os.html
def load_images_and_labels(folder_path):
    images = []
    labels = []
    sub_folders = os.listdir(folder_path)

    for sub_folder in sub_folders:
        label = sub_folder # In here is where the name of the sub folders is taken as the label
        image_files = os.listdir(os.path.join(folder_path, sub_folder))

        for image_file in image_files:
            image_path = os.path.join(folder_path, sub_folder, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_COLOR) # It reads the images in BGR
            if image is not None:
                image = cv2.resize(image, (128, 128))  # Redimention the images
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # This is to convert from BGR to RGB
                # Reference for this: https://docs.opencv.org/4.x/d4/da8/group__imgcodecs.html
                images.append(image)
                labels.append(label)

    return np.array(images), labels

In [ ]:
# To load images and labels
images, labels = load_images_and_labels(dataset_folder)

In [ ]:
# To create the training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.3, random_state=42, stratify=labels)

In [ ]:
# To crerate a dictionary that assigns a numeric value to each label
label_map = {label: idx for idx, label in enumerate(set(train_labels))}
# For both training and testing dataset, it replace the label with the correspodning numeric value stablich in the line above
train_labels_numeric = [label_map[label] for label in train_labels]
test_labels_numeric = [label_map[label] for label in test_labels]

In [ ]:
# Convert labels to one-hot encoding. Reference from: https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
num_classes = len(label_map)
train_labels_encoded = to_categorical(train_labels_numeric, num_classes=num_classes)
test_labels_encoded = to_categorical(test_labels_numeric, num_classes=num_classes)

In [ ]:
# To normalize and to convert in arrays of NumPy. Reference from: https://numpy.org/doc/stable/reference/generated/numpy.array.html
train_images = train_images / 255.0
test_images = test_images / 255.0
# To verify that the asignation of the values was made correctly
print("Mapping of labels to numeric values:")
print(label_map)

Mapping of labels to numeric values:
{'joy': 0, 'melancholy': 1, 'liveliness': 2, 'sadness': 3}


In [ ]:
# This was an extra added to improve the generalisability of the model, generating new modified versions of the images
datagen = ImageDataGenerator(
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True
)

datagen.fit(train_images)

In [ ]:
# Stablish the structure of the CNN model. Used a reference for most of the structure of the models
# the project of: https://medium.com/@skillcate/emotion-detection-model-using-cnn-a-complete-guide-831db1421fael
def build_feature_extractor(activation = 'relu', dropout_rate = 0.5, l2_rate = 0.001):
    input = Input(shape = (128, 128, 3))

    conv1 = Conv2D(64, (3, 3), padding = 'same', strides = (1, 1), kernel_regularizer = l2(l2_rate))(input)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation(activation)(conv1)
    pool1 = MaxPooling2D(pool_size = (2, 2))(conv1)
    pool1 = Dropout(dropout_rate)(pool1)

    conv2 = Conv2D(128, (3, 3), padding = 'same', strides = (1, 1), kernel_regularizer = l2(l2_rate))(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation(activation)(conv2)
    pool2 = MaxPooling2D(pool_size = (2, 2))(conv2)
    pool2 = Dropout(dropout_rate)(pool2)

    conv3 = Conv2D(256, (3, 3), padding = 'same', strides = (1, 1), kernel_regularizer = l2(l2_rate))(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation(activation)(conv3)
    pool3 = MaxPooling2D(pool_size = (2, 2))(conv3)
    pool3 = Dropout(dropout_rate)(pool3)

    conv4 = Conv2D(512, (3, 3), padding = 'same', strides = (1, 1), kernel_regularizer = l2(l2_rate))(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation(activation)(conv4)
    pool4 = MaxPooling2D(pool_size = (2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)

    flatten = Flatten()(pool4)
    feature_output = Dense(256, activation = activation)(flatten)

    feature_model = Model(inputs = input, outputs = feature_output)
    return feature_model

In [ ]:
feature_extractor = build_feature_extractor() # To show the structure of the model

In [ ]:
# This is to extract the characteristic of the images. Reference: https://scikit-learn.org/stable/modules/feature_extraction.html
train_features = feature_extractor.predict(train_images)
test_features = feature_extractor.predict(test_images)

33/33 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step


In [ ]:
# I make instance of StandarScaler to normalise the data. Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()
# This adjust the scaler. Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)
# This line of code creates the SVM model
svm_model = SVC()

In [ ]:
# To stablish the diferent hyperparameters values
svm_parameters = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]
}

In [ ]:
# Run the grid search with the previous mentioned hyperparameters
svm_grid_search = GridSearchCV(svm_model, svm_parameters, cv = 3, verbose = 2, n_jobs = -1)
svm_grid_search.fit(train_features, train_labels_numeric)

# To print results of best combination
print("Best SVM parameters:", svm_grid_search.best_params_)
print("Best SVM cross-validation score:", svm_grid_search.best_score_)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best SVM parameters: {'C': 1, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf'}
Best SVM cross-validation score: 0.48937759802269404
